# Lending Club
We will be working with financial lending data from Lending Club. Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return.

Most investors use a portfolio strategy to invest small amounts in many loans, with healthy mixes of low, medium, and high interest loans. We will focus on the mindset of a conservative investor who only wants to invest in loans that have a good chance of being paid off on time.

We will build machine learning models that can predict if a borrower will pay off their loan on time, and select one that suits the needs of the conservative investor.

To minimize initial loading, the following columns have been removed from the [source](https://www.lendingclub.com/info/download-data.action) file:

- **desc**: which contains a long text explanation for each loan; and
- **url**: which contains a link to each loan on Lending Club which can only be accessed with an investor account.

In [0]:
import pandas as pd

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/sharontan/machine-learning/master/Loans/LoanStats3a.csv',  low_memory=False)
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648.0,83.70%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687.0,9.40%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,NaN,2.0,0.0,2956.0,98.50%,10.0,f,0.0,0.0,3005.666844,3005.67,2400.00,605.67,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,NaN,10.0,0.0,5598.0,21%,37.0,f,0.0,0.0,12231.890000,12231.89,10000.00,2214.92,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Fully Paid,n,other,Personal,972xx,OR,17.94,0.0,Jan-96,0.0,38.0,NaN,15.0,0.0,27783.0,53.90%,38.0,f,0.0,0.0,4066.908161,4066.91,3000.00,1066.91,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.shape

(42542, 142)

## Data Cleaning

Many columns in this dataset consist of data that are derived from other columns while other columns leak data from the future. These columns would result in an overfitting of the training model.  There are also columns that provide no additional insight to the final decision. Columns that should be removed from the dataset:
- columns with more than 50% missing values
- columns with no additional insight:
  1. **id**: randomly generated field by Lending Club for unique identification purposes only
  2. **member_id**: also a randomly generated field by Lending Club for unique identification purposes only
  3. **emp_title**: not fundamentally useful without many other columns
- leaks data from the future (after the loan already started to be paid off)
  1. **funded_amnt**
  2. **funded_amnt_inv** 
  3. **issue_d**
  4. **out_prncp**
  5. **out_prncp_inv**
  6. **total_pymnt**
  7. **total_pymnt_inv**
  8. **total_rec_prncp**
  9. **total_rec_int**
  10. **total_rec_late_fee**
  11. **recoveries**
  12. **collection_recovery_fee**
  13. **last_pymnt_d**
  14. **last_pymnt_amnt**
- derived or redundant columns
  1. **grade**: results in the fixing of **int_rate**. We will use **int_rate** instead 
  2. **sub_grade**: also  results in the fixing of **int_rate**. We will use **int_rate** instead 
  3. **zip_code**: redundant with the **addr_state** column


### Drop columns

In [4]:
#drop columns with more than 50% missing values
data = data.dropna(thresh=(data.shape[0]/2), axis=1)
data.shape

(42542, 52)

In [5]:
#drop remaining columns listed above
cols_to_drop = ['funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 'issue_d', 'zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp', 'total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d', 'last_pymnt_amnt']
data = data.drop(cols_to_drop, axis=1)
data.shape

(42542, 34)

### Selecting Target Column

From the description of the columns from the [data dictionary](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097), we have selected **loan_status** as the target column as it is the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower.

In [6]:
data['loan_status'].value_counts()

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64

We are only interested in loans that are *Fully Paid* or *Charged Off*. We will drop the rest of the rows from the dataset. We will convert *Fully Paid* to 1 and *Charged Off* to 0.

In [0]:
interested = (data['loan_status'] == 'Fully Paid') | (data['loan_status'] == 'Charged Off')
data = data[interested]

In [8]:
conversion = {'Fully Paid': 1, 'Charged Off': 0}
data['loan_status'] = data['loan_status'].replace(conversion)
data['loan_status'].value_counts()

1    34116
0     5670
Name: loan_status, dtype: int64

### Drop columns with only one unique value

In [9]:
non_unique = []

for col in data.columns:
  non_null = data[col].dropna().unique()
  if len(non_null) < 2:
    non_unique.append(col)
    
data = data.drop(non_unique, axis=1)
data.shape

(39786, 24)

## Feature Engineering

In order to use the columns for sklearn models, we have to convert these columns to contain non-null numeric values. We may need to drop more columns or perform some string manipulation to convert them to float or integer types.

In [10]:
null_counts = data.isnull().sum()
null_counts

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1078
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
debt_settlement_flag       0
dtype: int64

In [11]:
data['emp_length'].value_counts()

10+ years    8899
< 1 year     4590
2 years      4394
3 years      4098
4 years      3444
5 years      3286
1 year       3247
6 years      2231
7 years      1775
8 years      1485
9 years      1259
Name: emp_length, dtype: int64

Filling the null values with a mean of the years of employment would skew the data with the category of *10+ years*.  Dropping the null rows would be a better strategy since we would only be losing 2.7% of the data.

In [12]:
data['revol_util'].value_counts()

0%        980
0.20%      63
63%        62
40.70%     59
66.70%     58
0.10%      58
31.20%     57
46.40%     57
70.40%     57
61%        57
66.60%     57
37.60%     56
65.90%     56
64.80%     55
49.80%     55
57.40%     55
48.90%     55
25.50%     55
32%        54
78.70%     54
68.60%     54
27.20%     54
35.30%     54
75.50%     54
64.60%     54
89.50%     54
51.60%     54
51.10%     54
76.60%     54
46.60%     54
         ... 
10.17%      1
0.05%       1
24.66%      1
0.04%       1
69.98%      1
8.01%       1
29.53%      1
58.77%      1
17.78%      1
70.94%      1
39.95%      1
28.41%      1
56.26%      1
9.34%       1
0.12%       1
5.33%       1
0.83%       1
8.58%       1
32.71%      1
54.22%      1
10.08%      1
33.29%      1
36.94%      1
49.63%      1
0.03%       1
18.82%      1
0.01%       1
26.32%      1
21.59%      1
7.28%       1
Name: revol_util, Length: 1089, dtype: int64

There are only 50 null values in the **revol_util** column. Dropping these rows will have little impact to the resulting dataset.

In [13]:
data['last_credit_pull_d'].value_counts()

May-19    7919
Oct-16    3579
Apr-19     926
Mar-19     766
Aug-18     674
Jan-19     651
Feb-19     628
Feb-17     593
Jul-18     583
Oct-18     577
Nov-18     525
Dec-18     507
Feb-13     501
Sep-18     433
May-18     386
Mar-13     386
Jan-18     358
Mar-16     350
Apr-18     345
Oct-17     341
Mar-18     335
Nov-17     327
Dec-14     324
Mar-14     320
Aug-17     318
Sep-14     315
Jul-14     313
Feb-18     310
Feb-14     309
Oct-14     301
          ... 
Dec-09      26
Jan-10      24
Jan-13      23
Oct-09      21
Nov-09      18
Aug-07      15
Aug-09      14
Feb-09      14
Apr-09      14
Jul-09      13
Jun-09      12
Jun-07      12
Jan-09      11
May-09      10
Mar-09       9
Aug-08       8
Sep-09       8
Oct-08       7
Dec-08       5
Sep-08       5
Mar-08       4
Jan-08       3
Oct-07       3
Feb-08       2
Sep-07       2
Jun-08       1
Dec-07       1
Jul-07       1
May-07       1
Jul-08       1
Name: last_credit_pull_d, Length: 141, dtype: int64

**emp_length**, **revol_util** and **last_credit_pull_d** are columns that provide valuable information on the loans and should not be dropped. Instead, we will drop the rows with null values and lose a small percentage of the data.

In [14]:
data = data.dropna()
data.shape

(37953, 24)

In [15]:
data['title'].value_counts()

Debt Consolidation                          2127
Debt Consolidation Loan                     1691
Personal Loan                                620
Consolidation                                499
debt consolidation                           483
Home Improvement                             345
Credit Card Consolidation                    342
Debt consolidation                           318
Small Business Loan                          317
Credit Card Loan                             307
Personal                                     295
Consolidation Loan                           252
Home Improvement Loan                        239
personal loan                                212
Wedding Loan                                 207
Loan                                         205
personal                                     201
consolidation                                197
Car Loan                                     195
Other Loan                                   179
Wedding             

The data quality of the **title** column  is not ideal for our analysis as we can see that it contains *Consolidation*, *Debt Consolidation*, *Debt Consolidation Loan* etc that mean the same thing. There are also varying versions of personal loans. Additionally, the **purpose** column contains no null values and convey similar information on the loan. Therefore, the **title** column should be dropped.

In [0]:
data = data.drop(['title'], axis=1)

In [17]:
data['pub_rec_bankruptcies'].value_counts()

0.0    36380
1.0     1566
2.0        7
Name: pub_rec_bankruptcies, dtype: int64

Since 96% of the values in **pub_rec_bankruptcies** are made up of one value. We can drop this column since it will not provide very much additional insights on our analysis.

In [18]:
data = data.drop(['pub_rec_bankruptcies'], axis=1)
data.shape

(37953, 22)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37953 entries, 0 to 39749
Data columns (total 22 columns):
loan_amnt               37953 non-null float64
term                    37953 non-null object
int_rate                37953 non-null object
installment             37953 non-null float64
emp_length              37953 non-null object
home_ownership          37953 non-null object
annual_inc              37953 non-null float64
verification_status     37953 non-null object
loan_status             37953 non-null int64
purpose                 37953 non-null object
addr_state              37953 non-null object
dti                     37953 non-null float64
delinq_2yrs             37953 non-null float64
earliest_cr_line        37953 non-null object
inq_last_6mths          37953 non-null float64
open_acc                37953 non-null float64
pub_rec                 37953 non-null float64
revol_bal               37953 non-null float64
revol_util              37953 non-null object
total_acc

The remaining columns do not consist of null values. Next, we will have to handle the columns that are not numeric. There are 11 object columns that needs to be considered before proceeding further.

In [20]:
data['term'].value_counts()

 36 months    27538
 60 months    10415
Name: term, dtype: int64

We can remove the string *months* and convert this column to an integer type.

In [21]:
data['term'] = data['term'].str.strip().str.slice(stop=2).astype(int)
data['term'].value_counts()

36    27538
60    10415
Name: term, dtype: int64

In [22]:
data['int_rate'].value_counts()

10.99%    934
13.49%    817
11.49%    800
7.51%     756
7.88%     701
7.49%     633
11.71%    591
9.99%     583
7.90%     559
5.42%     524
11.99%    519
12.69%    480
10.37%    454
12.42%    432
12.99%    431
8.49%     430
10.65%    421
6.03%     413
5.79%     390
8.90%     384
14.27%    382
7.29%     379
6.62%     376
11.86%    375
9.63%     368
10.59%    352
9.91%     349
5.99%     329
12.53%    327
7.14%     326
         ... 
16.08%      2
15.38%      2
15.76%      2
13.93%      1
12.49%      1
17.03%      1
24.59%      1
13.84%      1
21.48%      1
16.20%      1
18.72%      1
20.52%      1
24.40%      1
9.83%       1
11.22%      1
13.30%      1
22.64%      1
16.01%      1
9.51%       1
10.28%      1
10.46%      1
16.96%      1
12.36%      1
10.91%      1
16.33%      1
10.64%      1
17.54%      1
9.01%       1
17.46%      1
17.44%      1
Name: int_rate, Length: 337, dtype: int64

In [23]:
#remove '%' from the column to convert them to float
data['int_rate'] = data['int_rate'].str.strip().str[:-1].astype(float)
data['int_rate'].value_counts()

10.99    934
13.49    817
11.49    800
7.51     756
7.88     701
7.49     633
11.71    591
9.99     583
7.90     559
5.42     524
11.99    519
12.69    480
10.37    454
12.42    432
12.99    431
8.49     430
10.65    421
6.03     413
5.79     390
8.90     384
14.27    382
7.29     379
6.62     376
11.86    375
9.63     368
10.59    352
9.91     349
5.99     329
12.53    327
7.14     326
        ... 
15.38      2
15.07      2
16.08      2
22.64      1
24.40      1
21.48      1
10.46      1
16.96      1
9.01       1
12.49      1
20.52      1
17.54      1
13.84      1
10.64      1
13.30      1
18.72      1
17.46      1
10.28      1
9.83       1
17.03      1
24.59      1
12.36      1
9.51       1
13.93      1
16.01      1
10.91      1
16.33      1
16.20      1
11.22      1
17.44      1
Name: int_rate, Length: 337, dtype: int64

In [24]:
#remove 'years' from column and convert it to int
import re
data['emp_length'] = data['emp_length'].apply(lambda x : re.sub('[a-z\s\+\<]+', "", x)).astype(int)
data['emp_length'].value_counts()

10    8778
1     7558
2     4305
3     4033
4     3390
5     3246
6     2195
7     1749
8     1458
9     1241
Name: emp_length, dtype: int64

In [25]:
data['home_ownership'].value_counts()

RENT        18091
MORTGAGE    16991
OWN          2775
OTHER          96
Name: home_ownership, dtype: int64

We will convert this to categorical datatype and add dummy variables to convert them to numerical columns. 

In [0]:
categorical_cols = ['home_ownership']

In [27]:
data['verification_status'].value_counts()

Not Verified       15773
Verified           12376
Source Verified     9804
Name: verification_status, dtype: int64

In [0]:
#add to categorical columns list
categorical_cols.append('verification_status')

In [29]:
data['purpose'].value_counts()

debt_consolidation    17971
credit_card            4906
other                  3720
home_improvement       2836
major_purchase         2090
small_business         1730
car                    1480
wedding                 915
medical                 659
moving                  548
house                   364
vacation                345
educational             294
renewable_energy         95
Name: purpose, dtype: int64

In [0]:
#add to categorical columns list
categorical_cols.append('purpose')

In [31]:
data['addr_state'].value_counts()

CA    6825
NY    3624
FL    2727
TX    2628
NJ    1792
IL    1475
PA    1472
VA    1345
GA    1333
MA    1275
OH    1173
MD    1009
AZ     820
WA     784
CO     746
NC     730
CT     724
MI     679
MO     653
MN     584
NV     479
SC     456
OR     430
WI     428
AL     426
LA     417
KY     321
OK     292
KS     258
UT     247
AR     232
DC     209
RI     195
NM     178
WV     168
HI     168
NH     159
DE     108
MT      79
WY      78
AK      78
SD      61
VT      52
MS      19
TN      10
ID       4
IA       1
NE       1
IN       1
Name: addr_state, dtype: int64

There are too many discrete values to convert to dummy variables. Therefore, we will drop this column.

In [0]:
too_many_discrete = ['addr_state']

In [33]:
data['earliest_cr_line'].value_counts()

Nov-98    361
Oct-99    358
Dec-98    335
Oct-00    331
Dec-97    311
Nov-00    310
Nov-99    310
Oct-98    294
Nov-97    293
Sep-00    288
Dec-99    280
Dec-95    279
Jan-00    274
Jul-00    268
Dec-00    266
Aug-00    260
Nov-96    260
Dec-96    260
Sep-01    258
Oct-01    253
Oct-02    253
Sep-99    252
Aug-98    252
Apr-00    248
May-00    248
Aug-99    245
Oct-03    240
Oct-97    237
Jan-01    236
Sep-98    235
         ... 
Sep-56      1
Dec-65      1
Nov-68      1
Jun-68      1
Oct-08      1
Apr-69      1
Jun-67      1
Feb-67      1
Nov-54      1
May-72      1
Dec-50      1
Nov-72      1
Sep-63      1
Oct-70      1
Aug-67      1
Oct-68      1
Feb-66      1
May-67      1
Oct-64      1
Oct-63      1
Jun-72      1
Jun-69      1
Jan-46      1
Jul-67      1
Feb-73      1
Feb-69      1
Apr-64      1
Jan-63      1
Nov-62      1
Dec-66      1
Name: earliest_cr_line, Length: 514, dtype: int64

In [0]:
#add to too many discrete columns
too_many_discrete.append('earliest_cr_line')

In [35]:
data['revol_util'].value_counts()

0%        908
0.20%      60
40.70%     57
66.60%     56
61%        56
63%        56
70.40%     56
66.70%     55
64.80%     55
46.40%     55
31.20%     55
0.10%      55
37.60%     55
32%        54
68.70%     53
46.60%     53
70.80%     53
78.70%     53
57.40%     53
27.20%     53
68.60%     53
48.90%     53
76.60%     52
49.80%     52
41%        52
51.60%     52
64.60%     52
62.80%     52
65.90%     52
45.60%     52
         ... 
8.49%       1
62.31%      1
46.74%      1
33.39%      1
39.95%      1
24.89%      1
49.69%      1
26.32%      1
24.63%      1
16.02%      1
21.72%      1
32.71%      1
43.61%      1
37.73%      1
10.61%      1
24.65%      1
0.86%       1
24.66%      1
33.26%      1
5.33%       1
8.58%       1
18.82%      1
7.43%       1
0.54%       1
5.79%       1
25.74%      1
70.26%      1
0.03%       1
33.14%      1
70.94%      1
Name: revol_util, Length: 1087, dtype: int64

In [36]:
data['revol_util'] = data['revol_util'].str.strip().str[:-1].astype(float)
data['revol_util'].unique()

array([8.370e+01, 9.400e+00, 9.850e+01, ..., 4.963e+01, 4.000e-02,
       7.280e+00])

In [37]:
data['last_credit_pull_d'].value_counts()

May-19    7635
Oct-16    3423
Apr-19     874
Mar-19     732
Aug-18     644
Jan-19     624
Feb-19     607
Jul-18     567
Feb-17     561
Oct-18     546
Nov-18     498
Dec-18     493
Feb-13     492
Sep-18     415
May-18     372
Mar-13     367
Jan-18     348
Oct-17     336
Mar-16     335
Apr-18     333
Mar-18     321
Nov-17     320
Mar-14     309
Aug-17     309
Dec-14     308
Feb-18     300
Sep-14     297
Jul-14     297
Feb-14     293
Dec-17     290
          ... 
Jul-10      50
Oct-10      46
Nov-10      42
Aug-10      34
Feb-10      33
Mar-10      32
Apr-10      28
May-10      27
Jun-10      25
Jan-13      21
Dec-09      20
Jan-10      19
Oct-09      19
Nov-09      14
Apr-09      10
Jul-09      10
Aug-09      10
Aug-07       9
Feb-09       8
May-09       7
Mar-09       7
Sep-09       7
Jun-09       6
Oct-08       4
Mar-08       4
Jan-09       4
Dec-08       3
Sep-08       3
Aug-08       3
Jun-08       1
Name: last_credit_pull_d, Length: 132, dtype: int64

In [0]:
#add to too many discrete columns
too_many_discrete.append('last_credit_pull_d')

In [39]:
data['debt_settlement_flag'].value_counts()

N    37814
Y      139
Name: debt_settlement_flag, dtype: int64

In [0]:
#replace Y/N with numerical values
data['debt_settlement_flag'] = data['debt_settlement_flag'].replace({'N': 0, 'Y':1}).astype(int)

In [0]:
data.drop(too_many_discrete, axis=1, inplace=True)

In [42]:
data.dtypes.value_counts()

float64    12
int64       4
object      3
dtype: int64

We will now convert the remaining 3 object columns to categorical.

In [43]:
dummy_df = pd.get_dummies(data[categorical_cols])
dummy_df.shape

(37953, 21)

In [0]:
loans = pd.concat([data, dummy_df], axis=1)

In [45]:
loans.drop(categorical_cols, axis=1, inplace=True)
loans.dtypes.value_counts()

uint8      21
float64    12
int64       4
dtype: int64

## Predictive Modelling

Since we are in the mindset of a conservative investor, identifying false positives is more crucial than false negatives (possible missed opportunities). Therefore, we should ultimately select a model that fits this criteria. 

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

### Split dataframe into features and target columns

In [0]:
target = loans['loan_status']

In [0]:
features = loans.drop(['loan_status'], axis=1)

In [49]:
target.value_counts()

1    32648
0     5305
Name: loan_status, dtype: int64

### Random Forest Classifier

This dataset consists of 85% positive values i.e where loan_status is *Fully Paid*. We will have to add weights to this model to "balance" the samples.

In [0]:
rfc = RandomForestClassifier(class_weight='balanced')
predictions = cross_val_predict(rfc, features, target, cv=10)

In [0]:
predictions = pd.Series(predictions)

In [0]:
tn, fp, fn, tp = confusion_matrix(target, predictions).ravel()

In [53]:
true_positive_rate = float(tp) / (tp+fn)
true_positive_rate

0.977854692477334

In [54]:
false_positive_rate = float(fp) / (tn+fp)
false_positive_rate

0.9293119698397738

This model is not ideal as the false positive rate is too high. 

### Logistic Regression

Like the Random Forest model, we will have to add weights to the Logistic Regression model to \"balance\" the samples.

In [0]:
lr = LogisticRegression(class_weight='balanced')
predictions = cross_val_predict(lr, features, target, cv=10)

In [0]:
predictions = pd.Series(predictions)

In [0]:
tn, fp, fn, tp = confusion_matrix(target, predictions).ravel()

In [58]:
true_positive_rate = float(tp) / (tp+fn)
true_positive_rate

0.6577125704484195

In [59]:
false_positive_rate = float(fp) / (tn+fp)
false_positive_rate

0.37474081055607916

We are still getting a 37% false positive rate. We will add a heavier weight for loan status of 0 to assign a heavier penalty for wrong classification. Given that the positive loan status is about 6 times of the negative loan status, *balanced* would give roughly 6 to the negative loan status when we used the *balanced* in *class_weights*. We will double it for the next model and assess the results.



In [0]:
weights = {0: 12, 1: 1}
lr = LogisticRegression(class_weight=weights)
predictions = cross_val_predict(lr, features, target, cv=10)

In [0]:
predictions = pd.Series(predictions)

In [0]:
tn, fp, fn, tp = confusion_matrix(target, predictions).ravel()

In [63]:
true_positive_rate = float(tp) / (tp+fn)
true_positive_rate

0.18212447929429063

In [64]:
false_positive_rate = float(fp) / (tn+fp)
false_positive_rate

0.05504241281809614

A conservative investor would favor this false positive rate. 

## Conclusion

A conservative investor would likely be comfortable with the Logistic Regression model with adjusted weights to balance this heavily positive dataset. However, the trade-off is that this investor would miss out on many opportunities where the loan would be paid back.